In [108]:
##Initialize Network

In [109]:
#Has three parameters: the number of inputs (number of features), the number of neurons to have in the hidden layer,
#and the number of outputs
def initialize_network(n_inputs,n_hidden, n_outputs):
    network = list()
    #At each neuron in the hidden layer, it has n_inputs+1 weights that is initially randomized between 0 to 1
    hidden_layer = [{'weights': [random() for i in range(n_inputs+1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    #At each neuron in the hidden layer, it has n_hidden+1 weights (correspond to each neuron in the hidden layer) 
    #that is initially randomized between 0 to 1
    output_layer = [{'weights': [random() for i in range(n_hidden+1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

In [110]:
#An example to create a small network
from random import seed
from random import random

seed(1)
network = initialize_network(2,1,2)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


In [111]:
##Forward Propagate

In [112]:
#Neuron Activation
#Calculate the activation of one neuron given an input
def activate(weights, inputs):
    activation = weights[-1] #Bias
    for i in range(len(weights)-1):
        activation += weights[i]*inputs[i]
    return activation

In [113]:
#Neuron Transfer
#Transform the values from the neuron to output
#For example sigmoid function
def sigmoid(activation):
    return 1.0/(1.0+exp(-activation))

In [114]:
#Forward propagate input to a network output
#Input: network, row(data)
def forward_propagate(network, row):
    inputs = row #Data as the inputs to the first hidden layer
    for layer in network:
        new_inputs = []
        #Iterate through every layer
        for neuron in layer:
            activation = activate(neuron['weights'], inputs) #Calculates activation at each neuron in the layer
            neuron['output'] = sigmoid(activation) #Calculates output from the activation at each neuron
            new_inputs.append(neuron['output']) #The outputs is used as the inputs to the next layer
        inputs = new_inputs
    return inputs

In [115]:
#Example to test forward propagation
from math import exp

seed(1)
network = initialize_network(2,1,2)
row = [1,0] #Example of an arbitrary input
output = forward_propagate(network, row)
print(output)

[0.6629970129852887, 0.7253160725279748]


In [116]:
##Back Propagate Error

In [117]:
#Transfer Derivative
#Calculate slope from an output value of a neuron
def transfer_derivative_sigmoid(output):
    return output*(1.0-output)

In [118]:
#Error Backpropagation
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i!= len(network)-1: #For the hidden layer
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i+1]:
                    error += (neuron['weights'][j]*neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)): #Iterate through neurons at output layer
                neuron = layer[j]
                errors.append(expected[j]-neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j]*transfer_derivative_sigmoid(neuron['output'])

In [119]:
#Example to test back propagation
network = initialize_network(2,1,2)
row = [1,0] #Example of an arbitrary input
output = forward_propagate(network, row)
expected = [0,1]
backward_propagate_error(network, expected)
for layer in network:
    print(layer)

[{'weights': [0.7887233511355132, 0.0938595867742349, 0.02834747652200631], 'output': 0.6936142046010635, 'delta': -0.011477619712406795}]
[{'weights': [0.8357651039198697, 0.43276706790505337], 'output': 0.7335023968859138, 'delta': -0.1433825771158816}, {'weights': [0.762280082457942, 0.0021060533511106927], 'output': 0.6296776889933221, 'delta': 0.08635312555373359}]


In [120]:
##Train Network
#The network is trained using stochastic gradient descent.
#Process: Forward propagating the input, backward propagating the error, updating the network weights
#Divided into two parts: Update weights and Train Network

In [121]:
#Update Weights
def update_weights(network, row, learning_rate):
    #Iterate through layers
    for i in range(len(network)): 
        inputs = row[:-1]
        #Except the first hidden layer
        if i!=0: 
            inputs = [neuron['output'] for neuron in network[i-1]] #The inputs is the output from the previous layer
        for neuron in network[i]: #Iterate through neurons
            for j in range(len(inputs)):
                   neuron['weights'][j] += learning_rate*neuron['delta']*inputs[j]
            neuron['weights'][-1] += learning_rate*neuron['delta'] #For bias

In [122]:
#Train network
def train_network(network, train, learning_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, learning_rate)
        print('>Epoch=%d, learning_rate=%.3f, error=%.3f' %(epoch, learning_rate, sum_error))

In [131]:
#Example to learn simple dataset
seed(1)
dataset = [[2.7810836,2.550537003,0],
    [1.465489372,2.362125076,0],
    [3.396561688,4.400293529,0],
    [1.38807019,1.850220317,0],
    [3.06407232,3.005305973,0],
    [7.627531214,2.759262235,1],
    [5.332441248,2.088626775,1],
    [6.922596716,1.77106367,1],
    [8.675418651,-0.242068655,1],
    [7.673756466,3.508563011,1]]
n_inputs = len(dataset[0])-1 #Number of features
n_outputs = len(set([row[-1] for row in dataset])) #Number of class
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)
for layer in network:
    print(layer)

>Epoch=0, learning_rate=0.500, error=6.350
>Epoch=1, learning_rate=0.500, error=5.531
>Epoch=2, learning_rate=0.500, error=5.221
>Epoch=3, learning_rate=0.500, error=4.951
>Epoch=4, learning_rate=0.500, error=4.519
>Epoch=5, learning_rate=0.500, error=4.173
>Epoch=6, learning_rate=0.500, error=3.835
>Epoch=7, learning_rate=0.500, error=3.506
>Epoch=8, learning_rate=0.500, error=3.192
>Epoch=9, learning_rate=0.500, error=2.898
>Epoch=10, learning_rate=0.500, error=2.626
>Epoch=11, learning_rate=0.500, error=2.377
>Epoch=12, learning_rate=0.500, error=2.153
>Epoch=13, learning_rate=0.500, error=1.953
>Epoch=14, learning_rate=0.500, error=1.774
>Epoch=15, learning_rate=0.500, error=1.614
>Epoch=16, learning_rate=0.500, error=1.472
>Epoch=17, learning_rate=0.500, error=1.346
>Epoch=18, learning_rate=0.500, error=1.233
>Epoch=19, learning_rate=0.500, error=1.132
[{'weights': [-1.4688375095432327, 1.850887325439514, 1.0858178629550297], 'output': 0.029980305604426185, 'delta': -0.00595466041

In [ ]:
##Prediction

In [132]:
#Function to calculate prediction
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

In [133]:
for row in dataset:
    prediction = predict(network, row)
    print('Expected = %d, Predicted = %d' %(row[-1],prediction))

Expected = 0, Predicted = 0
Expected = 0, Predicted = 0
Expected = 0, Predicted = 0
Expected = 0, Predicted = 0
Expected = 0, Predicted = 0
Expected = 1, Predicted = 1
Expected = 1, Predicted = 1
Expected = 1, Predicted = 1
Expected = 1, Predicted = 1
Expected = 1, Predicted = 1
